## 【問題1】クロスバリデーション
事前学習期間は検証データを分割しておき、それに対して指標値を計算することで検証を行っていました。しかし、分割の仕方により精度は変化します。実践的には クロスバリデーション を行います。<br>
具体的には分割を複数回行い、それぞれに対して学習と検証を行う方法です。複数回の分割を行う関数はscikit-learnにKFoldとして用意されています。<br>

In [26]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor # 回帰用
from sklearn.ensemble import RandomForestClassifier # 分類用
from sklearn.metrics import auc
from sklearn.model_selection import KFold


# ファイル名（パス）を指定する
csv_path = "E:/DiveIntoCode/source/application_train.csv" # 絶対パス

# 指数表示の禁止を設定する ※numpyのみなので注意
np.set_printoptions(suppress=True)

# csvファイル読み込み ※該当のcsvの１行目が列名となっているため、列名を指定したりせず、そのまま読み込む
df_train = pd.read_csv(csv_path)


In [27]:
# Xに説明変数、yに目的変数となる列をそれぞれ抽出して代入
df_X = df_train[["AMT_CREDIT","DAYS_EMPLOYED"]]
df_y = df_train["TARGET"]
# X,yをndarrayへ変換
X = df_X.values
y = df_y.values

# test_splitにX、yを代入（デフォルト(指定なし)で訓練75%、検証25%となる）
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# indexをランダムに分ける関数の模様
# n_splits=? の?個にデータを分割(全データ÷?)する。検証とテストデータの割合は分割する数による ５分割であれば４(検証)：１(テスト)
# 結果、分割されたデータ群はテストデータにそれぞれ１回ずつなる
kf = KFold(n_splits=5)
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # インスタンス化
    scaler = StandardScaler()

    # 訓練用のデータのみを.fitで標準化
    scaler.fit(X_train)

    # 訓練用、検証用双方のデータにtransformで標準化
    tr_X_tarin = scaler.transform(X_train)
    tr_X_test = scaler.transform(X_test)

    # 学習（ランダムフォレスト）
    regr = RandomForestClassifier(max_depth=2, random_state=0)
    regr.fit(tr_X_tarin, y_train)

# 学習モデルによる推定
Y_pred = regr.predict(tr_X_test)

# ROC曲線とAUCの出力
fpr, tpr, thresholds = roc_curve(y_test, Y_pred)
result_auc = auc(fpr, tpr)

print(result_auc)

TRAIN: [ 61503  61504  61505 ... 307508 307509 307510] TEST: [    0     1     2 ... 61500 61501 61502]
TRAIN: [     0      1      2 ... 307508 307509 307510] TEST: [ 61503  61504  61505 ... 123002 123003 123004]
TRAIN: [     0      1      2 ... 307508 307509 307510] TEST: [123005 123006 123007 ... 184504 184505 184506]
TRAIN: [     0      1      2 ... 307508 307509 307510] TEST: [184507 184508 184509 ... 246006 246007 246008]
TRAIN: [     0      1      2 ... 246006 246007 246008] TEST: [246009 246010 246011 ... 307508 307509 307510]
0.5


## 【問題2】グリッドサーチ
これまで分類器のパラメータは基本的にデフォルトの設定を使用していました。パラメータの詳細は今後のSprintで学んでいくことになりますが、パラメータは状況に応じて最適なものを選ぶ必要があります。パラメータを探索するために グリッドサーチ と呼ばれる総当たり的手法が一般的に利用されます。<br>
グリッドサーチをパイプラインの中に組み込みましょう。<br>

In [39]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor # 回帰用
from sklearn.ensemble import RandomForestClassifier # 分類用
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV


# ファイル名（パス）を指定する
csv_path = "E:/DiveIntoCode/source/application_train.csv" # 絶対パス

# 指数表示の禁止を設定する ※numpyのみなので注意
np.set_printoptions(suppress=True)

# csvファイル読み込み ※該当のcsvの１行目が列名となっているため、列名を指定したりせず、そのまま読み込む
df_train = pd.read_csv(csv_path)


In [42]:
# Xに説明変数、yに目的変数となる列をそれぞれ抽出して代入
df_X = df_train[["AMT_CREDIT","DAYS_EMPLOYED"]]
df_y = df_train["TARGET"]
# X,yをndarrayへ変換
X = df_X.values
y = df_y.values

### ランダムフォレストで使用するパラメータ
# n_estimators 10, 50, 100, 150, 200
# max_depth : 2, 4, 6, 8, 10
# min_samples_split : 1, 3, 5, 7, 9
# max_leaf_nodes : 5, 10, 15, 20, 25
# min_samples_leaf : 2, 4, 6, 8, 10
# max_sample（ブートストラップサンプル）
# max_features

#グリッドサーチ対象のハイパーパラメーターを準備
# param_grid = {
#     'n_estimators': [10, 50, 100, 150, 200],
#     'max_depth': [2, 4, 6, 8, 10],
#     'min_samples_split': [1, 3, 5, 7, 9],
#     'min_samples_leaf': [2, 4, 6, 8, 10],
#     'max_leaf_nodes': [5, 10, 15, 20, 25]
#     }
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [2, 10],
    }

roop_cnt = 0

# クロスバリデーション
n_num = 2 # データの分割数

kf = KFold(n_splits=n_num)
for train_index, test_index in kf.split(df_X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # インスタンス化
    scaler = StandardScaler()

    # 訓練用のデータのみを.fitで標準化
    scaler.fit(X_train)

    # 訓練用、検証用双方のデータにtransformで標準化
    tr_X_tarin = scaler.transform(X_train)
    tr_X_test = scaler.transform(X_test)

    # グリッドサーチ
    forest_grid = GridSearchCV(estimator=RandomForestClassifier(random_state=42),
                               param_grid = param_grid,
                               n_jobs = 1,
                               cv = 5,
                               scoring = "roc_auc"
                              )
    forest_grid.fit(tr_X_tarin, y_train)

    roop_cnt += 1
    print(roop_cnt)

    # 学習モデルによる推定
    Y_pred = forest_grid.predict(tr_X_test)

    # ROC曲線とAUCの出力
    fpr, tpr, thresholds = roc_curve(y_test, Y_pred)
    result_auc = auc(fpr, tpr)

    print(result_auc)

TRAIN: [153756 153757 153758 ... 307508 307509 307510] TEST: [     0      1      2 ... 153753 153754 153755]
1
0.49999291859929895
TRAIN: [     0      1      2 ... 153753 153754 153755] TEST: [153756 153757 153758 ... 307508 307509 307510]
2
0.4999964657067526


## 【問題3】Kernelからの調査
KaggleのKernelから自身にはなかったアイデアを見つけ出して、列挙してください。そして、効果があると考えられるものを検証してください。<br>

メモリの節約：速度面では処理が増えた分マイナスになるが、処理で使用するメモリを省略することでより大規模なデータに対応する際に効果が期待できそう

```
def reduce_mem_usage(df_):
    start_mem = df_.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe: {:.2f} MB'.format(start_mem))
    
    for c in df_.columns[df_.dtypes != 'object']:
        col_type = df_[c].dtype
        
        c_min = df_[c].min()
        c_max = df_[c].max()
        if str(col_type)[:3] == 'int':
            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                df_[c] = df_[c].astype(np.int8)
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                df_[c] = df_[c].astype(np.int16)
            elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                df_[c] = df_[c].astype(np.int32)
            else:
                df_[c] = df_[c].astype(np.int64)  
        else:
            if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                df_[c] = df_[c].astype(np.float16)
            elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                df_[c] = df_[c].astype(np.float32)
            else:
                df_[c] = df_[c].astype(np.float64)

    end_mem = df_.memory_usage().sum() / 1024**2
    print('Memory usage after optimization: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df_
```

In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor # 回帰用
from sklearn.ensemble import RandomForestClassifier # 分類用
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV


# ファイル名（パス）を指定する
csv_path = "E:/DiveIntoCode/source/application_train.csv" # 絶対パス

# 指数表示の禁止を設定する ※numpyのみなので注意
np.set_printoptions(suppress=True)

# csvファイル読み込み ※該当のcsvの１行目が列名となっているため、列名を指定したりせず、そのまま読み込む
df_train = pd.read_csv(csv_path)

# Xに説明変数、yに目的変数となる列をそれぞれ抽出して代入
df_X = df_train[["AMT_CREDIT","DAYS_EMPLOYED"]]
df_y = df_train["TARGET"]
# X,yをndarrayへ変換
X = df_X.values
y = df_y.values

### ランダムフォレストで使用するパラメータ
# n_estimators 10, 50, 100, 150, 200
# max_depth : 2, 4, 6, 8, 10
# min_samples_split : 1, 3, 5, 7, 9
# max_leaf_nodes : 5, 10, 15, 20, 25
# min_samples_leaf : 2, 4, 6, 8, 10
# max_sample（ブートストラップサンプル）
# max_features

#グリッドサーチ対象のハイパーパラメーターを準備
# param_grid = {
#     'n_estimators': [10, 50, 100, 150, 200],
#     'max_depth': [2, 4, 6, 8, 10],
#     'min_samples_split': [1, 3, 5, 7, 9],
#     'min_samples_leaf': [2, 4, 6, 8, 10],
#     'max_leaf_nodes': [5, 10, 15, 20, 25]
#     }
param_grid = {
    'n_estimators': [100],
    'max_depth': [6],
    'min_samples_split': [5],
    'min_samples_leaf': [6],
    'max_leaf_nodes': [15]
    }

roop_cnt = 0

# クロスバリデーション
n_num = 5 # データの分割数

kf = KFold(n_splits=n_num)
for train_index, test_index in kf.split(df_X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # インスタンス化
    scaler = StandardScaler()

    # 訓練用のデータのみを.fitで標準化
    scaler.fit(X_train)

    # 訓練用、検証用双方のデータにtransformで標準化
    tr_X_tarin = scaler.transform(X_train)
    tr_X_test = scaler.transform(X_test)

    # グリッドサーチ
    forest_grid = GridSearchCV(estimator=RandomForestClassifier(random_state=42),
                               param_grid = param_grid,
                               n_jobs = 1,
                               cv = 5,
                               scoring = "roc_auc"
                              )
    forest_grid.fit(tr_X_tarin, y_train)

    roop_cnt += 1
    print(roop_cnt)

    # 学習モデルによる推定
    Y_pred = forest_grid.predict(tr_X_test)

    # ROC曲線とAUCの出力
    fpr, tpr, thresholds = roc_curve(y_test, Y_pred)
    result_auc = auc(fpr, tpr)

    print(result_auc)


TRAIN: [ 61503  61504  61505 ... 307508 307509 307510] TEST: [    0     1     2 ... 61500 61501 61502]
1
0.5
TRAIN: [     0      1      2 ... 307508 307509 307510] TEST: [ 61503  61504  61505 ... 123002 123003 123004]
2
0.5
TRAIN: [     0      1      2 ... 307508 307509 307510] TEST: [123005 123006 123007 ... 184504 184505 184506]
3
0.5
TRAIN: [     0      1      2 ... 307508 307509 307510] TEST: [184507 184508 184509 ... 246006 246007 246008]
4
0.5
TRAIN: [     0      1      2 ... 246006 246007 246008] TEST: [246009 246010 246011 ... 307508 307509 307510]
5
0.5
